# 0. Loading data.

## 0.1 Reading CSVs

In [12]:
import pandas as pd

df = pd.read_csv("../../data/source/Moriartyred.csv",
                  usecols=["UUID","ActionType"])

df2 = pd.read_csv("/home/itsquall/Descargas/T4.csv", 
                  usecols=['UUID', 'CPU_0', 'CPU_1', 'CPU_2', 'CPU_3', 'Traffic_TotalRxBytes',
       'Traffic_TotalTxBytes',
       'Traffic_TotalTxPackets', 'Traffic_TotalWifiRxBytes',
       'Traffic_TotalWifiRxPackets', 'Traffic_TotalWifiTxBytes',
       'Traffic_TotalWifiTxPackets', 'Battery_current_avg', 'Battery_health',
       'Battery_level', 'Battery_online', 'Battery_plugged', 'Battery_present',
       'Battery_scale', 'Battery_status', 'Battery_temperature',
       'Battery_voltage', 'MemTotal', 'MemFree', 'Buffers'])


/home/itsquall/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (34) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## 0.2 Merging datasets by UUID

In [13]:
df['UUID'] = pd.to_datetime(df['UUID'], unit="ms")
df['UUID'] = df['UUID'].dt.round('t')


df2['UUID'] = pd.to_datetime(df['UUID'], unit="ms")
df2['UUID'] = df['UUID'].dt.round('t')

data = pd.merge(df,df2, on=['UUID'])

## 0.3 Replacing and cleaning data

In [14]:
data['ActionType'] = data['ActionType'].replace(['malicious'], 1)
data['ActionType'] = data['ActionType'].replace(['benign'], 0)
y = data['ActionType']

data = data.drop('UUID', 1)
data = data.drop('ActionType', 1)

# 1. Principal Component Analysis

## 1.1. Scalation

In [6]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler()
datanorm = scaler.fit_transform(data)

## 1.2 Modelling (PCA)

In [7]:
from sklearn.decomposition import PCA
n_components = 12
estimator = PCA(n_components)
X_pca = estimator.fit_transform(datanorm)

print(estimator.explained_variance_ratio_.cumsum())

[0.28443482 0.45159281 0.58935699 0.70984932 0.7732553  0.82682327
 0.87210232 0.91177916 0.94358338 0.97324863 0.99219339 0.99764723]


## 1.3 Plotting (PCA)

In [8]:
import numpy
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

n_components = 2
estimator_plot = PCA(n_components)
X_pca_plot = estimator.fit_transform(datanorm)

fig = plt.figure()
plt.scatter(X_pca_plot[:, 0], X_pca_plot[:, 1], c=y.values)
ax = Axes3D(fig)
x_pca = X_pca_plot[:,0]
y_pca = X_pca_plot[:,1]
z_pca = X_pca_plot[:,2]
ax.scatter(x_pca,y_pca,z_pca, c=y.values)
plt.show()

<Figure size 640x480 with 2 Axes>

# 2. Training Model (KNN)

In [9]:
from sklearn.metrics import accuracy_score
from sklearn import neighbors
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.3)

In [10]:
# Create color maps
from matplotlib.colors import ListedColormap

cmap_light = ListedColormap(['#FFAAAA', '#AAFFAA', '#AAAAFF'])
cmap_bold = ListedColormap(['#FF0000', '#00FF00', '#0000FF'])

In [11]:
from sklearn import neighbors

for weights in ['uniform', 'distance']:
    n_neighbors = 1
    # Instance of Neighbours Classifier and fit the data.
    clf = neighbors.KNeighborsClassifier(n_neighbors, weights=weights)
    clf.fit(X_train,y_train)
    
    
    
    y_pred = clf.predict(X_test)
    
    
    print(accuracy_score(y_test, y_pred)*100)

92.3076923076923
92.3076923076923
